In [1]:
import gymnasium as gym
import torch
from skrl.envs.wrappers.torch import wrap_env
from skrl.memories.torch import RandomMemory
from skrl.trainers.torch import SequentialTrainer, ParallelTrainer
from skrl.utils import set_seed
from reward import simple_reward
from preprocess import preprocess
from lr_schedulers import CosineAnnealingWarmUpRestarts

set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="mc_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

[skrl:INFO] Seed: 42


In [2]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/train/month_15m/**/**/*.pkl",
    preprocess=preprocess,
    reward_function=simple_reward,
    positions=[-1, 1],
    multiplier=[1, 2, 5],
    trading_fees=0.01,
    borrow_interest_rate=0.03,
    portfolio_initial_value=1000,
    max_episode_duration="max",
    verbose=1,
    window_size=60,
)

In [4]:
env = gym.make(**env_cfg)
# env = gym.wrappers.FlattenObservation(env)
obs = env.observation_space
env = gym.make_vec(
    vectorization_mode="sync",
    num_envs=2,
    wrappers=[gym.wrappers.FlattenObservation],
    **env_cfg,
)
env = wrap_env(env, wrapper="gymnasium")

[skrl:INFO] Environment wrapper: gymnasium


In [5]:
device = env.device
replay_buffer_size = 1024 * 1 * env.num_envs
memory_size = int(replay_buffer_size / env.num_envs)
memory = RandomMemory(memory_size=memory_size, num_envs=env.num_envs, device=device, replacement=False)

In [ ]:
import torch.nn as nn
from skrl.models.torch import DeterministicMixin, MultiCategoricalMixin, Model
from skrl.utils.spaces.torch import unflatten_tensorized_space
import torch.nn.functional as F


class Shared(MultiCategoricalMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        unnormalized_log_prob=True,
        reduction="sum",
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultiCategoricalMixin.__init__(self, unnormalized_log_prob, reduction)
        DeterministicMixin.__init__(self, clip_actions)

        self.net_feature = nn.Sequential(
            nn.Conv1d(8, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.RReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.RReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(64, 128, kernel_size=3, stride=2),
            nn.RReLU(),
            nn.Flatten(),
             # torch.Size([batch, 1144])
            nn.Linear(896, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.GELU(),
        )
        self.net_info = nn.Sequential(
            nn.Linear(3, 4),
            nn.BatchNorm1d(4),
            nn.RReLU(),
            nn.Dropout(0.2),
            nn.Linear(4, 8),
            nn.RReLU(),
        )

        self.mean_layer = nn.Linear(32 + 8, self.num_actions)
        self.log_std_parameter = nn.Parameter(torch.zeros(self.num_actions))

        self.value_layer = nn.Linear(32 + 8, 1)

    def act(self, inputs, role):
        if role == "policy":
            return MultiCategoricalMixin.act(self, inputs, role)
        elif role == "value":
            return DeterministicMixin.act(self, inputs, role)
        
    def compute(self, inputs, role):
        states = unflatten_tensorized_space(obs, inputs["states"])  # 32, 288, 7

        if role == "policy":
            features = self.net_feature(states["features"].permute(0, 2, 1))
            info = torch.cat([states["total_ROE"], states["position"], states["ROE"]], dim=1)
            info = self.net_info(info)
            fusion = torch.cat([features, info], dim=1)

            self._shared_output = fusion
            action = self.mean_layer(self._shared_output)
            return action, {}
        elif role == "value":
            if self._shared_output is None:
                features = self.net_feature(states["features"].permute(0, 2, 1))
                info = torch.cat([states["total_ROE"], states["position"], states["ROE"]], dim=1)
                info = self.net_info(info)
                fusion = torch.cat([features, info], dim=1)

                shared_output = fusion
            else:
                shared_output = self._shared_output

            self._shared_output = None
            value = self.value_layer(shared_output)
            return value, {}

In [ ]:
models = {}
models["policy"] = Shared(env.observation_space, env.action_space, device)
models["value"] = models["policy"]

for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)

In [ ]:
from skrl.agents.torch.ppo import PPO_DEFAULT_CONFIG

cfg = PPO_DEFAULT_CONFIG.copy()
cfg["rollouts"] = memory_size
cfg["learning_epochs"] = 64
cfg["mini_batches"] = 4
cfg["discount_factor"] = 0.99
cfg["learning_rate"] = 0
cfg["learning_starts"] = 0
cfg["learning_rate_scheduler"] = CosineAnnealingWarmUpRestarts
cfg["learning_rate_scheduler_kwargs"] = {
    "T_0": 16 * cfg["learning_epochs"],
    "T_mult": 2,
    "T_up": cfg["learning_epochs"],
    "eta_max": 1e-3,
    "gamma": 0.8,
}

cfg["experiment"]["write_interval"] = 1000 * 8
cfg["experiment"]["checkpoint_interval"] = 100000
cfg["experiment"]["directory"] = "runs/torch/mddt"

In [ ]:
import warnings
from skrl.agents.torch.ppo import PPO

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

agent = PPO(
    models=models,
    memory=memory,
    cfg=cfg,
    observation_space=env.observation_space,
    action_space=env.action_space,
    device=device,
)
agent.load("/home/pitin/Desktop/hp/runs/torch/mddt/24-12-04_13-40-56-176824_PPO/checkpoints/best_agent.pt")
# agent.load("/home/pitin/Desktop/hp/runs/torch/mddt/24-11-27_17-35-07-776155_PPO/checkpoints/best_agent.pt")
# agent.load("runs/torch/mddt/24-11-26_13-28-22-043499_PPO/checkpoints/best_agent.pt")
cfg_trainer = {"timesteps": 10000000, "headless": True, "environment_info": "pc_counter"}
trainer = ParallelTrainer(cfg=cfg_trainer, env=env, agents=[agent])

In [ ]:
# trainer.eval()

In [ ]:
states, infos = env.reset()
timestep = 0
timesteps = 3000
terminated = torch.tensor([[False] * env.num_envs])

import time, pprint

while (not terminated.any()):
    time.sleep(0.5)
    agent.pre_interaction(timestep=timestep, timesteps=timesteps)

    with torch.no_grad():
        actions = agent.act(states, timestep=timestep, timesteps=timesteps)[0]
        # actions = torch.tensor([[0, 0]], device='cuda:0')
        # print(actions)
        next_states, rewards, terminated, truncated, infos = env.step(actions)
        pprint.pprint(infos)
        env.render()

    super(type(agent), agent).post_interaction(timestep=timestep, timesteps=timesteps)

    if env.num_envs > 1:
        states = next_states
    else:
        if terminated.any() or truncated.any():
            with torch.no_grad():
                states, infos = env.reset()
        else:
            states = next_states
